In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
combinations=[(True,True, True),(True,True,False),(True,False,True),(True,False, False),(False,True, True),(False,True, False),(False, False,True),(False,False, False)]#expand this set for more variables
variable={'P':0,'Q':1, 'R':2}#expand this set matching combinations indices for variables
#set of rules
kb=''#should be a cnf 
q=''#should be a cnf
priority={'~':3,'v':1,'^':2}

In [ ]:
def input_rules():
    global kb,q
    kb=input("Enter rule : ")
    q=input("Enter the Query : ")

In [ ]:
def entailment():
    global kb,q
    print("*"*10+"Truth Table Reference"+"*"*10)
    print("P  ","  Q  ","   R   ","  KB  ",' Alpha')
    print("*"*20)
    for comb in combinations:
        s=evaluatePostfix(toPostfix(kb),comb)
        f=evaluatePostfix(toPostfix(q),comb)
        a,b,c=comb
        print(a,b,c,s,f)
        print("-"*10)
        if s and not f:
            return False
    return True

In [ ]:
def isOperand(c):
    return c.isalpha() and c!='v'
def isLeftParenthesis(c):
    return c=='('
def isRightParenthesis(c):
    return c==')'
def isEmpty(stack):
    return len(stack)==0
def peek(stack):
    return stack[-1]
def hasLessOrEqualPriority(c1,c2):
    try: return priority[c1]<=priority[c2]
    except KeyError : return False

In [ ]:
def toPostfix(infix):
    stack=[]
    postfix=''
    for c in infix:
        if isOperand(c):
            postfix+=c
        else:
            if isLeftParenthesis(c):
                stack.append(c)
            elif isRightParenthesis(c):
                operator =stack.pop()
                while (not isLeftParenthesis(operator)):
                    postfix+=operator
                    operator=stack.pop()
                    
            else:
                while (not isEmpty(stack)) and hasLessOrEqualPriority(c,peek(stack)):
                    postfix+=stack.pop()
                stack.append(c)
    while (not isEmpty(stack)):
        postfix +=stack.pop()
    return postfix

In [ ]:
def evaluatePostfix(exp,comb):
    stack=[]
    for i in exp:
        if isOperand(i):
            stack.append(comb[variable[i]])
        elif i=="~":
            val1=stack.pop()
            stack.append(not val1)
        else:
            val1=stack.pop()
            val2=stack.pop()
            stack.append(_eval(i,val2,val1))
    return stack.pop()

In [ ]:
def _eval(i,val1,val2):
    if i=='^': return val2 and val1 
    return val2 or val1
input_rules()
ans=entailment()
if ans: print("The Knowledgebase entails Query .")
else: print("The Knowledgebase doesnt enatils Query.")